# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [165]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [166]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [167]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [168]:
df_total_q = data.groupby(['CustomerID','ProductName']).Quantity.sum().reset_index()
df_total_q.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [169]:
df_piv_prod = pd.pivot_table(df_total_q,columns='CustomerID', index='ProductName', values='Quantity', aggfunc=np.sum).fillna(0)
df_piv_prod.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [181]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
pd_mat_sim = pd.DataFrame(squareform(pdist(df_piv_prod.T, metric='euclidean'), force='no', checks=True),
                          columns = df_piv_prod.T.index, index = df_piv_prod.T.index)
pd_mat_sim.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [182]:
def top5_cust(customer):
    similar_customers = list(pd_mat_sim[[customer]].nsmallest(5+1, customer)[1:6].index)
    return(similar_customers)

customer = 33
l_top5_cust = top5_cust(customer)
print(l_top5_cust)

[264, 3317, 3535, 2503, 3305]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [183]:
similar_customers = df_total_q[df_total_q.CustomerID.isin(l_top5_cust)]
similar_customers.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [184]:
products_sim_cust = similar_customers.groupby('ProductName').Quantity.sum().sort_values(ascending=False).reset_index()
products_sim_cust.head()

,ProductName,Quantity
0,Butter - Unsalted,3
1,Soup - Campbells Bean Medley,3
2,Towels - Paper / Kraft,3
3,Wine - Ej Gallo Sierra Valley,3
4,"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [185]:
df_merged_prod = df_piv_prod.merge(products_sim_cust, how='left', on='ProductName')
df_merged_prod[df_merged_prod[customer]==0].sort_values(by=['Quantity'], ascending=False).head(5)

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
415,Wine - Blue Nun Qualitatswein,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,3.0
341,Soup - Campbells Bean Medley,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0,3.0
421,Wine - Ej Gallo Sierra Valley,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,50.0,0.0,25.0,0.0,0.0,0.0,0.0,3.0
73,Butter - Unsalted,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,3.0
448,Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,2.0


In [97]:
df_total_q[(df_total_q['CustomerID']==customer)]#.ProductName.value_counts()
# pd_mat_sim[[customer]].reset_index().CustomerID
#.size
# df_total_q[df_total_q['CustomerID']==customer]

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
5,33,"Beef - Chuck, Boneless",1
6,33,Beef - Prime Rib Aaa,1
7,33,Beer - Original Organic Lager,1
8,33,Beer - Rickards Red,1
9,33,Black Currants,1


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [186]:
dict_recomendations = {}
customers = df_total_q.CustomerID.unique()

for customer in customers:
    similar_customers = list(pd_mat_sim[[customer]].nsmallest(5+1, customer)[1:6].index)
    similar_customers = df_total_q[df_total_q.CustomerID.isin(similar_customers)]
    products_sim_cust = similar_customers.groupby('ProductName').Quantity.sum().sort_values(ascending=False).reset_index()
    df_merged_prod = df_piv_prod.merge(products_sim_cust, how='left', on='ProductName')
    df_merged_prod = df_merged_prod[df_merged_prod[customer]==0].sort_values(by=['Quantity'], ascending=False).head(5)
    dict_recomendations[customer] = list(df_merged_prod['ProductName'].values)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [187]:
df_recomendations=pd.DataFrame(dict_recomendations).T
df_recomendations.head()

,0,1,2,3,4
33,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,Wine - Ej Gallo Sierra Valley,Butter - Unsalted,Yoghurt Tubes
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Wanton Wrap,Bay Leaf,Pork - Kidney
264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,"Veal - Inside, Choice",Bread - Italian Roll With Herbs,Fondant - Icing
356,Butter - Unsalted,"Veal - Inside, Choice","Nut - Chestnuts, Whole","Beets - Candy Cane, Organic",Raspberries - Fresh
412,"Sauce - Gravy, Au Jus, Mix",Sprouts - Baby Pea Tendrils,Olive - Spread Tapenade,Wine - Blue Nun Qualitatswein,Cheese - Cambozola


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [190]:
pd_mat_sim = pd.DataFrame(squareform(pdist(df_piv_prod.T, metric='correlation'), force='no', checks=True),
                          columns = df_piv_prod.T.index, index = df_piv_prod.T.index)
dict_recomendations = {}
customers = df_total_q.CustomerID.unique()

for customer in customers:
    similar_customers = list(pd_mat_sim[[customer]].nsmallest(5+1, customer)[1:6].index)
    similar_customers = df_total_q[df_total_q.CustomerID.isin(similar_customers)]
    products_sim_cust = similar_customers.groupby('ProductName').Quantity.sum().sort_values(ascending=False).reset_index()
    df_merged_prod = df_piv_prod.merge(products_sim_cust, how='left', on='ProductName')
    df_merged_prod = df_merged_prod[df_merged_prod[customer]==0].sort_values(by=['Quantity'], ascending=False).head(5)
    dict_recomendations[customer] = list(df_merged_prod['ProductName'].values)
df_recomendations=pd.DataFrame(dict_recomendations).T
df_recomendations.head()

,0,1,2,3,4
33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Veal - Osso Bucco,Crush - Cream Soda
200,Otomegusa Dashi Konbu,Potatoes - Idaho 100 Count,Crackers - Trio,Milk Powder,Pail With Metal Handle 16l White
264,"Water - Mineral, Natural",Wine - Toasted Head,Snapple - Iced Tea Peach,Hersey Shakes,Garbag Bags - Black
356,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet",Ocean Spray - Kiwi Strawberry,Cheese - Cheddarsliced,Soup Knorr Chili With Beans
412,Butter - Unsalted,Cake - Mini Cheesecake,"Salmon - Atlantic, Skin On",Wine - Hardys Bankside Shiraz,Gloves - Goldtouch Disposable
